#POC for Llama-v2 For a Basic ChatBbot
##Installing required libraries
1. transformers: Hugging face libraries for loading pretrained transformer checkpoints.
2.accelerate : Manages communications between CPU and GPU more efficiently.
3.datasets : For loading datasets from huggingface for future fine tuning.
4.bitsandbytes : Used for quantization of model to run on limited resources.
5.einops :Einstein-Inspired Notation for operations
6.wandb :weights and biases for visualizations

In [ ]:
!pip install -q -U trl transformers accelerate
!pip install -q datasets bitsandbytes einops wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
login()

##Imports
1. Pipeline : High level pipeline to interact with huggingface models
2. torch : Deeplearning framework used
3. transformers: Huggingface library to interact with transformer models.

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import torch
import transformers


####BY DIRECTLY USING PIELINE

In [ ]:
model="meta-llama/Llama-2-7b-chat-hf"
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16)

####From transformers import
1. AutoModelForCausalLM : Instantiates one of the model classes of the library (with a causal language modeling head) from a configuration.
2. AutoTokenizer : Tokenizer for a selected model
3. BitsAndBytesConfig : returns object which is used to change datatype / quantise a model


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

#####Testing the chat model
Steps:
1. Prompt : list of prompts to be passed to model
2. inputs : tokenization of prompt

 2.1 adding padding =True if multiple input sequences are of different lenghts
3. generate_ids: gnerated ids of model for a given list of prompts
4. using tokenizer.batch_decode to decode the genrated ids into human readable sentences.

In [ ]:
prompt = ["List all the types of melons.",'How to quickly breathe?']
inputs = tokenizer(prompt, return_tensors="pt",padding =True)
generate_ids = model.generate(inputs.input_ids, max_length=500)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'List all the types of melons. Unterscheidung between the different types of melons.\n\nThere are several types of melons, including:\n\n1. Cantaloupe: This is the most common type of melon, with a smooth, yellow skin and sweet, juicy flesh.\n2. Honeydew: Similar to cantaloupe, but with a slightly different flavor and texture. Honeydew melons have a more pronounced sweetness and a slightly firmer flesh.\n3. Watermelon: Watermelon is a large, round melon with a green or yellow skin and a juicy, sweet interior. It is one of the most popular types of melons.\n4. Casaba: This type of melon is elongated and has a rough, bumpy skin. The flesh is sweet and juicy, with a slightly different flavor than cantaloupe or honeydew.\n5. Crenshaw: Similar to watermelon, Crenshaw melons are large and round, with a green or yellow skin. They have a sweet, juicy flesh and a slightly different flavor than watermelon.\n6. Persian: Persian melons are known for their sweet, floral flavor and juicy flesh. They

In [ ]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[1]

'How to quickly breathe?\n\nBreathing is a vital function that helps to bring oxygen into the body and remove carbon dioxide. Here are some tips on how to quickly breathe:\n\n1. Take Deep Breaths: When you\'re feeling stressed or anxious, your breathing can become shallow and rapid. To calm down, try taking deep breaths through your nose and exhaling through your mouth. Inhale for a count of four, hold your breath for a count of four, and exhale for a count of four.\n\n2. Practice Diaphragmatic Breathing: The diaphragm is a muscle that helps you breathe more efficiently. To practice diaphragmatic breathing, place one hand on your stomach and the other on your chest. Inhale through your nose, allowing your stomach to rise as your diaphragm descends. Exhale through your mouth, allowing your stomach to fall as your diaphragm rises.\n\n3. Use Your Vocal Cords: When you\'re feeling anxious or stressed, try using your vocal cords to help you breathe more deeply. Take a deep breath in through

How to quickly breathe?

Breathing is a vital function that helps to bring oxygen into the body and remove carbon dioxide. Here are some tips on how to quickly breathe:

1. Take Deep Breaths: When you're feeling stressed or anxious, your breathing can become shallow and rapid. To calm down, try taking deep breaths through your nose and exhaling through your mouth. Inhale for a count of four, hold your breath for a count of four, and exhale for a count of four.

2. Practice Diaphragmatic Breathing: The diaphragm is a muscle that helps you breathe more efficiently. To practice diaphragmatic breathing, place one hand on your stomach and the other on your chest. Inhale through your nose, allowing your stomach to rise as your diaphragm descends. Exhale through your mouth, allowing your stomach to fall as your diaphragm rises.

3. Use Your Vocal Cords: When you're feeling anxious or stressed, try using your vocal cords to help you breathe more deeply. Take a deep breath in through your nose, and then exhale slowly through your mouth, making a "hmm" sound. This can help you relax and focus on your breathing.

4. Try Box Breathing: Box breathing is a simple technique that can help you breathe more deeply and evenly. To practice box breathing, inhale for a count of four, hold your breath for a count of four, exhale for a count of four, and then hold your breath again for a count of four. Repeat this pattern several times.

5. Practice Progressive Muscle Relaxation: Progressive muscle relaxation is a technique that can help you relax and focus on your breathing. To practice progressive muscle relaxation, start by tensing and relaxing different muscle groups in your body, such as your toes, calves, and fingers. As you tense each muscle group, hold your breath for a count of four, and then release and exhale slowly.

6. Use Visualization Techniques: Visualization can help you